# Get explainability scores

This notebook is used to get explainability attributions for previously trained models. It is using the same configuration as the [starter](starter.ipynb) notebook.

## Load data and model

You can load your already processed data with `load_data` and the `from_cleaned` argument (be sure to have specified the path for the cleaned data in the config file). The additional arguments need to be the same as the ones used for training, but if `from_cleaned` is `True`, they will be `None` by default.

You can load your trained model with `load_model` method with `.pt` file path as argument.

In [ ]:
import sys

sys.path.append("../")

import torch

from beexai.dataset.dataset import Dataset
from beexai.dataset.load_data import load_data
from beexai.explanation.explaining import CaptumExplainer

from beexai.explanation.plot_attr import bar_plot, plot_swarm, plot_waterfall
from beexai.training.train import Trainer
from beexai.utils.time_seed import set_seed
from beexai.utils.sampling import stratified_sampling

seed = 42
set_seed(seed)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

DATA_NAME = "kickstarter"
MODEL_NAME = "NeuralNetwork"
NUM_SAMPLES = 100
data_test, target_col, task, _ = load_data(
    from_cleaned=True, config_path=f"config/{DATA_NAME}.yml"
)
scale_params = {
    "x_num_scaler_name": "quantile_normal",
    "x_cat_encoder_name": "ordinalencoder",
    "y_scaler_name": "labelencoder",  # change to minmax or another float scaler for regression
    "cat_not_to_onehot": ["name"],
}
data = Dataset(data_test, target_col)
X_train, X_test, y_train, y_test = data.get_train_test(
    test_size=0.2, scaler_params=scale_params
)

NUM_LABELS = data.get_classes_num(task)
NN_PARAMS = {"input_dim": X_train.shape[1], "output_dim": NUM_LABELS}
trainer = Trainer(MODEL_NAME, task, NN_PARAMS, device)
trainer.load_model(f"../output/models/{DATA_NAME}/{MODEL_NAME}.pt")

For faster testing, we use the function `stratified_sampling` that samples a fraction of the data while keeping the same distribution of the target variable.

In [ ]:
X_test, y_test = stratified_sampling(X_test, y_test, NUM_SAMPLES, task)

## Get attributions

Explanation methods are based on Captum library including `ShapleyValueSampling`, `FeatureAblation`, `KernelSHAP`, `IntegratedGradients`, `InputXGradient`, `Saliency` and `DeepLift`. 

Be aware that Gradient based methods can't work with non Deep Learning models.

The method `compute_attributions` automatically computes the attributions for each sample in the test set and save them in the specified folder. If attributions are already computed in the folder, it will automatically load them.

In [ ]:
METHOD = "IntegratedGradients"
explainer = CaptumExplainer(
    trainer.model, task=task, method="IntegratedGradients", sklearn=False, device=device
)

explainer.init_explainer()
all_preds = trainer.model.predict(X_test.values)
attributions = explainer.compute_attributions(
    X_test,
    DATA_NAME,
    MODEL_NAME,
    METHOD,
    "../output/explain/",
    all_preds,
    save=True,
    use_abs=False,
)

Different plots similar to the ones provided by SHAP can be obtained for any kind of attributions. Be sure that the features names provided are in the same order as the ones used for training:
- `bar_plot` directly plots the attributions for a specific instance or the mean of the attributions for all instances in an histogram.
- `plot_waterfall` plots the attributions for a specific instance in a waterfall plot.
- `plot_swarm` plots the attributions for all instances in a swarm plot.

In [ ]:
features_names = list(data_test.columns)
features_names.remove(target_col)
bar_plot(attributions, feature_names=features_names, mean=True)
plot_waterfall(attributions[0], feature_names=features_names, mean=False)
plot_swarm(X_test, attributions, feature_names=features_names)

## Next steps

- Go to [Metric](metrics.ipynb) to get the metrics on the attributions you just computed